In [5]:
from Crypto.Cipher import AES
from Crypto.Random import random
from hashlib import sha256

ca_key = (123911229913422142875264410911959753429255609781060012540592865384481400430784339515602026823919372067245349130530370079412035001372557207523878469898937327849288805682100737656179353201361997603827751534685210816357362467822736133251729781421390068496293277151621728336136046748401714936280146300069794792289, 65537)

P = 90217665064898209874945440415607645619739552209209829698859302616784804764535733761793578496057231124307702060703777047113648993244620436073319613409562635176198907683514151487667596066485528729421061091099275256268502121631878101098253283229895343146346757918637948217208979264467987857303672675468072062419

G = 13

certs = ["416c696365_7d2e831fc7c37b4932604c66de52a3c7f631eee0badffb42f40899fb88ff19e41518fc8326f620aaeb54cecd8bd22a5fa8576848c131370c0fcb8fb0e093fbd0d76b73c47c04ffe2b913472595c9987d53b8cdffa4ff16f6a6176f215e6ded63e65089d763691a4cef91284e6934b95580e64854304b99e87215b582e201222d", "426f62_2453e551e17519b6dcfdae46f75b9d92dd943fcb5d2c71911adb83f43753dfba86be013dfb8c2413cbc70eac852e312040d3fbf471b57da2ae7a7aa95d39f3b972e34214f5cc69093d9929b2c5fc24684b0d5443dcb8338d1f52bb34f9913bd1e98c29302bbbd3cb532001adde7f06d93badcdee7b12b1b550a816537e950bad"]

def pad(msg):
    n = 16 - len(msg) % 16
    return msg + bytes([n]) * n

def unpad(msg):
    assert len(msg) > 0 and len(msg) % 16 == 0
    n = msg[-1]
    assert 1 <= n <= 16
    assert msg[-n:] == bytes([n]) * n
    return msg[:-n]

In [6]:
fromAliceAndBob1 = [
    "3d15d52459d2c64fa409725b58bb45783ce0083eb713470cd26b776d0753f944da4683d25e74ceb909c2cf6566fe759973a02e31a1237c2ba521eb29d69bc449ab6c082bd4f32c4aca3983bd1f3645058d43cb38334d694db44c818df344aa1aff04aeca932c05e9d7e2d50cd5a1f378b7f8a1a7f701aeeb71ae4ae2339da345",
    "459c616bc8a6b4206021198e082fb9df85fdd729dede5601cb7d09030858ed630f24fd0a36e89004b6d307de2013f15047c7a1508929e92abf7540db122dba727d720e606d0491273fddc6a1ef0b87ac426ccc23506d6703d3b66f18a87cef2e8e808370e15b3b4f90837c4f22bd7267a6477b46e2e8daea78e5b1b3dc4de5dd"
]

# myKey = random.getrandbits(1024)
myKey = 0
myKey -= myKey & 1

keys = [pow(int(from1, 16), myKey, P) for from1 in fromAliceAndBob1]
while keys[0] < 2**512:
    myKey += 1
    keys = [pow(int(from1, 16), myKey, P) for from1 in fromAliceAndBob1]
aesKeys = [int.to_bytes(key % (2**128), 16, "big") for key in keys]

toAliceAndBob1 = [hex(pow(G, myKey, P))[2: ] for _ in fromAliceAndBob1]

print("toAliceAndBob1 == {}".format(toAliceAndBob1))

toAliceAndBob1 == ['d', 'd']


In [7]:
fromAliceAndBob2 = [
    "1f1a6bac90a179c78409aec3d91ab2aba1c5cd74f133268fc4572ad1b0f296822c9a5d802a6eb7d3a03c0f7a86a143563ba6f99479b28f4fea58fa6efd406b7514a71ff0aa4671c91e861e13f87b4ee834621cbaab3877df78cdb85493fa2f8a7284404e2581c31887feff57ad8e36d2d86f5dec1c8bfafb9e2bc42ee671e3491d62688411612e762a0c4fac9f8038734461f374ecf3f531aa698dd1e34d51428b56b9512aae73972fa7724bc7e1d789189a8eab2f1a24115d4f6cc7a09a3d7776a2b7cc0b7566b14e01e8355108eb984d0674c5da344fa9072065d34d4b89cf6fcf8b343ec15cb6befdb133380f0cc3f92f10f25ccea3820f52a1b122700fbfd0e92f780d49d91c5b6fe48b2157f45c6fb833b6c1c4dade8937a52f6289a56d",
    "58ced58a0284863b06424364bf27bf54de70d45635090f7948d4e26711fec4cc879b6a8f0d5f6c917a6605894001a89b42adc5d9dcae9e06aa4285be3935b78971e55fc1efde9293e16a66b11cf239e55180680862c8a42c246f62b8bc9a6456f73e025b64582ffa133dceffac24eea19c2bf933bacc345093ac472d8778ad57c6b76586091724ac2b0fe6bc3331e35d9cfd3bf078bfe8c9257786f3480b0b51d90f6b9eeb64b3c075a7afb3e13f91c33f3f8ba4d8803b1834f55c19a3a1117037e2554a49f5f4a8ebb137d8b1cfd4609203bdd7dd25be6e113e3249dbe2a13620da2c394493933bab5a83485ea09e14de9d94f04d45b183aba4cf7255f0140beef725427f115cfaed0537e0b0d052315cadb5c70a164a4d3c50be0e86f1b478"
]

aesIvsAndCiphers = [bytes.fromhex(from2) for from2 in fromAliceAndBob2]
aesIvs = [aesIvAndCipher[: 16] for aesIvAndCipher in aesIvsAndCiphers]
ciphers = [aesIvAndCipher[16: ] for aesIvAndCipher in aesIvsAndCiphers]
aeses = [AES.new(aesKey, AES.MODE_CBC, aesIv) for aesKey, aesIv in zip(aesKeys, aesIvs)]

toAlice2 = [(aesIv + aes.encrypt(pad(cert.encode()))).hex() for aesIv, aes, cert in zip(aesIvs, aeses, reversed(certs))][0]

print("toAlice2 == {}".format(toAlice2))

toAlice2 == 1f1a6bac90a179c78409aec3d91ab2ab514b56af28564ea947f7c64fd38ae2d98b0f8f605fe1d8032b24f7c713cce8080959c7857dc5f584bac154892f062fc9245ced1d66e9d788cf7bad24a4fa95b1f2e61aad1294cd4179262875f66014c94455bb32f01bbcb1aecbb5f59594c137af535231fc20c3a5ec32e1c5751a929458d1851c75e0b7d652d4a4ad4933feec452a803e4560078ea6af348656a44c9a10428cf555ac26e6fcf3ed34d55d223de919a33d4f9d5a4b66163597cc1d31ab28314184968be8f281440588ba47c637b8cdf3cb6a6c6213ae5db8e709e3b7d1c871db5779656f5bf21ba27ea3581a0ac0fc065460252d372e54c087505f387c3f30242c7174852c5ba2bcd4fa06591336bd21941e3fab64679e6a10d3a4a0b1


In [8]:
fromAlice3 = "6e8207c71b0880da568ea563ca0e25a5c5d6b4e100d7d010046e59cb8c53da0b896072b9ec3d515db06ff822f5b6ca14d0d54b54c8b07ef564e6cdc7ed76325e"

aesIvAndCipher = bytes.fromhex(fromAlice3)
aesIv, cipher = aesIvAndCipher[: 16], aesIvAndCipher[16: ]
aes = AES.new(aesKeys[0], AES.MODE_CBC, aesIv)
message = unpad(aes.decrypt(cipher)).decode()
flag = message

print("flag == {}".format(flag))
print("key == {}".format(keys[0]))
print("(key < 2**512) == {}".format(keys[0] < 2**512))
print("flag is: {}".format("flag1" if keys[0] < 2**512 else "flag2"))

flag == flag{Wh4t_A_w3Ak_Pr0toCo1_4edd1383}
key == 42895544866691787986500722234248872981216321889752551445173584351912362610369144138045240248577142014758055301331878779572695427580690051334989645339079705220092520064000752265205474276296150476912104962453033711501398511345307551089036256943806897022927541462173552148968155707045399058774470120738771542853
(key < 2**512) == False
flag is: flag2


In [10]:
flag = "flag{Wh4t_A_w3Ak_Pr0toCo1_4edd1383}"
key = 42895544866691787986500722234248872981216321889752551445173584351912362610369144138045240248577142014758055301331878779572695427580690051334989645339079705220092520064000752265205474276296150476912104962453033711501398511345307551089036256943806897022927541462173552148968155707045399058774470120738771542853
(key < 2**512) == False
"flag is: flag2"

'flag is: flag2'